In [57]:
import selenium
from selenium import webdriver
import time
import pandas as pd

In [61]:
def get_drive_launch(url):
    driver = webdriver.Chrome('./driver/chromedriver.exe')
    driver.maximize_window()
    driver.implicitly_wait(10)
    driver.get(url)
    return driver


In [80]:
def get_highlevel_details_naukri(job, location, stop=None):
    url = 'https://www.naukri.com/'
    drivr = get_drive_launch(url)

    drivr.find_element_by_id("qsb-keyword-sugg").send_keys(job)
    drivr.find_element_by_id('qsb-location-sugg').send_keys(location)
    drivr.find_element_by_xpath("//div[@class='search-btn']/button").click()
    print(drivr.find_element_by_xpath("//div[@class='sortAndH1Cont']/h1").text)
    titles=[]
    companies=[]
    exps=[]
    salaries=[]
    locs=[]

    job_titles = drivr.find_elements_by_xpath("//div[@class='info fleft']/a")
    job_companies = drivr.find_elements_by_xpath("//div[@class='info fleft']/div/a[1]")
    job_exps = drivr.find_elements_by_xpath("//div[@class='info fleft']/ul/li[1]")
    job_salaries = drivr.find_elements_by_xpath("//div[@class='info fleft']/ul/li[2]")
    job_locations = drivr.find_elements_by_xpath("//div[@class='info fleft']/ul/li[3]")
    for job_title, job_company, job_exp, job_salary, job_loc in zip(job_titles[:stop],job_companies[:stop],job_exps[:stop],job_salaries[:stop],job_locations[:stop]):
        titles.append(job_title.text)
        companies.append(job_company.text)
        exps.append(job_exp.text)
        salaries.append(job_salary.text)
        locs.append(job_loc.text)
    
    return drivr, pd.DataFrame({'Job Title':titles, 'Company':companies, 'Experience Required':exps,'Salary offered':salaries, 'Location':locs})

## Q1: Write a python program to scrape data for “Data Analyst” Job position in “Bangalore” location. You have to scrape the job-title, job-location, company_name, experience_required. You have to scrape first 10 jobs data.


This task will be done in following steps:
1. first get the webpage https://www.naukri.com/
2. Enter “Data Analyst” in “Skill,Designations,Companies” field and enter “Bangalore” in “enter the location” field.
3. Then click the search button.
4. Then scrape the data for the first 10 jobs results you get.
5. Finally create a dataframe of the scraped data. 

Note- All of the above steps have to be done in code. No step is to be done manually

In [82]:
d, dataFrame = get_highlevel_details_naukri('Data Analyst', 'Bangalore', 10)
d.quit()
dataFrame

Data Analyst Jobs In Bangalore


,Job Title,Company,Experience Required,Salary offered,Location
0,Assistant Manager II - Data Analyst,Flipkart Internet Private Limited,4-8 Yrs,Not disclosed,Bangalore/Bengaluru
1,Data Analyst,Super India Tech Mark,0-2 Yrs,"1,25,000 - 2,25,000 PA.",Bangalore/Bengaluru(Devalapur)
2,Data Analyst,Super India Tech Mark,0-2 Yrs,"70,000 - 1,25,000 PA.",Bangalore/Bengaluru(Chintamani)
3,Data Analyst,tech mahindra ltd,4-8 Yrs,Not disclosed,Bangalore/Bengaluru
4,Data Analyst,GlaxoSmithKline Pharmaceuticals Limited,2-7 Yrs,Not disclosed,Bangalore/Bengaluru
5,Data Analyst,Xiaomi,2-6 Yrs,Not disclosed,Bangalore/Bengaluru
6,Hiring For Data Analyst,Concentrix Daksh Services India Private Limited.,2-7 Yrs,"4,00,000 - 9,00,000 PA.","Hyderabad/Secunderabad, Pune, Chennai, Bangalo..."
7,Senior Data Analyst,Cerner,3-5 Yrs,Not disclosed,Bangalore/Bengaluru
8,Data Analyst with Marketing Analytics-Capco,Capco Technologies Pvt Ltd,4-9 Yrs,"7,00,000 - 17,00,000 PA.","Pune, Bangalore/Bengaluru"
9,Business Data Analyst - MIS & Reporting,INTERTRUST GROUP,3-7 Yrs,Not disclosed,"Mumbai, Bangalore/Bengaluru"


## Q2: Write a python program to scrape data for “Data Scientist” Job position in “Bangalore” location. You have to scrape the job-title, job-location, company_name, full job-description. You have to scrape first 10 jobs data.


This task will be done in following steps:
1. first get the webpage https://www.naukri.com/
2. Enter “Data Scientist” in “Skill,Designations,Companies” field and enter “Bangalore” in “enter the location” field.
3. Then click the search button.
4. Then scrape the data for the first 10 jobs results you get.
5. Finally create a dataframe of the scraped data.

Note- 
1. All of the above steps have to be done in code. No step is to be done manually.
2. Please note that you have to scrape full job description. For that you may have to open each job separately as shown below

In [88]:
d, dataFrame = get_highlevel_details_naukri('Data Scientist', 'Bangalore', 10)
main_window = d.current_window_handle
all_windows = d.window_handles
print(main_window, all_windows)
for window in all_windows:
    if window != main_window:
        d.switch_to.window(window)
        d.close()
        
d.switch_to.window(main_window)
main_window = d.current_window_handle
all_windows = d.window_handles
print(main_window, all_windows)
full_descriptions = []

for element in d.find_elements_by_xpath("//div[@class='info fleft']/a")[:10]:
    element.click()
    time.sleep(2)
    all_windows = d.window_handles
    d.switch_to.window(all_windows[1])
    try:
        full_descriptions.append(d.find_element_by_xpath("//section[@class='job-desc']//p").text)
    except:
        full_descriptions.append(d.find_element_by_xpath("//div[@class='clearboth description']").text)
    d.close()
    d.switch_to.window(main_window)
d.quit()    

dataFrame['Full Description'] = full_descriptions
dataFrame.drop(['Experience Required', 'Salary offered'], axis=1, inplace = True)
dataFrame

Data Scientist Jobs In Bangalore
CDwindow-D1F1EC707E8EE9D32F97B23991976253 ['CDwindow-D1F1EC707E8EE9D32F97B23991976253', 'CDwindow-BE3AEECB26DC3C20C53DDA60C5477787', 'CDwindow-0B6B8EC3F34029169DE7858D3B213436', 'CDwindow-31B8ACDC68339B792B13FBD95CADA274']
CDwindow-D1F1EC707E8EE9D32F97B23991976253 ['CDwindow-D1F1EC707E8EE9D32F97B23991976253']


,Job Title,Company,Location,Full Description
0,Required- Data Scientist (NLP)-Axis Bank - 6 m...,Axis Bank Limited,"Kolkata, New Delhi, Hyderabad/Secunderabad, Pu...",Roles and Responsibilities - The bank generate...
1,Data Scientist - Python/ MATLAB/ Machine Learn...,Wrackle Technologies Pvt Ltd,Bangalore/Bengaluru,
2,Principal Data Scientist - Machine/Deep Learni...,Fidius advisory,Bangalore/Bengaluru,
3,Data Scientist/ Analyst,Becton Dickinson India Pvt. Ltd,Bangalore/Bengaluru,Roles and Responsibilities
4,Opportunity For Data Scientist Internship - Be...,Corner Stone Solutions,Bangalore/Bengaluru,Location - Bangalore / Bengaluru\nDuration- 6 ...
5,Data Scientist || Data Analyst || Data science,Inspiration Manpower Consultancy Pvt. Ltd.,"Navi Mumbai, Bangalore/Bengaluru",
6,Analytics & AI Product Mgmt - Sr. Data Scientist,GlaxoSmithKline Pharmaceuticals Limited,Bangalore/Bengaluru,GSK is one of the world’s foremost pharmaceuti...
7,Data Scientist: Machine Learning,Fractal Analytics,"Bangalore/Bengaluru, Delhi / NCR, Mumbai (All ...",Position Description:
8,"Senior Data Scientist, , Data Science",Visa Inc.,Bangalore/Bengaluru,We are seeking an innovative and analytical th...
9,Lead Data Scientist - Immediate Joiners are Re...,Techolution India Private Limited,"Mumbai, Hyderabad/Secunderabad, Pune, Chennai,...",We are looking for qualified Lead Data Scienti...


In [85]:
len(full_descriptions)

10